In [9]:
from manim import *
import difflib

[03/27/24 12:26:03] WARNING  Missing Plugins: {''}                                                   ]8;id=216657;file:///usr/local/lib/python3.11/site-packages/manim/plugins/__init__.py\__init__.py]8;;\:]8;id=192968;file:///usr/local/lib/python3.11/site-packages/manim/plugins/__init__.py#17\17]8;;\

In [14]:
def tokenize(code: Code) -> List[VMobjectFromSVGPath|Dot]:
    tokens = []
    for line in code[2]: # 0 = frame, 1 = dot, 2 = paragraph
        current_vmobject_group = []
        for vmobject in line:
            if isinstance(vmobject, Dot):
                if len(current_vmobject_group) > 0:
                    tokens.append(VGroup(*current_vmobject_group))
                    current_vmobject_group = []

                # vmobject is a brace or a dot
                tokens.append(vmobject)
                continue

            current_vmobject_group.append(vmobject)
            
        if len(current_vmobject_group) > 0:
            tokens.append(VGroup(*current_vmobject_group))

    return tokens

In [3]:
rust_code = """impl From<u64> for Health {
  fn from(bitfield: u64) -> Health {
    // ...
  }
}"""

In [12]:
%%manim -qm -v WARNING CodeBlock

class CodeBlock(Scene):
    def construct(self):
        impl_body = Code(code=rust_code, language="rust", insert_line_no=False)
        tokens = tokenize(impl_body)

        for token in tokens:
            token_anim = Write(token)
            self.play(ChangeSpeed(
                token_anim,
                speedinfo={0:3},
                rate_func=linear
            ))

Manim Community v0.18.0

In [5]:
rust_code = """impl From<u64> for Health {
  fn from(bitfield: u64) -> Health {
    // ...
  }
}"""

token_replacements = """impl TryFrom<u64> for Health {
  type Error = anyhow::Error;
  fn from(bitfield: u64) -> Result<Self, Self::Error> {
    // ...
  }
}"""


In [33]:
%%manim -qm -v WARNING CodeBlock

class CodeBlock(Scene):
    def construct(self):
        code = tokenize(Code(code=rust_code, language="rust", insert_line_no=False))
        replacement_code = tokenize(Code(code=token_replacements, language="rust", insert_line_no=False))
        for token in code:
            anim = Write(token)
            sped_up = ChangeSpeed(
                anim,
                speedinfo={0:3},
                rate_func=linear
            )
            self.play(sped_up)

        for token, replacement in zip(code, replacement_code):
            anim = Transform(token, replacement)
            sped_up = ChangeSpeed(
                anim,
                speedinfo={0:3},
                rate_func=linear
            )
            self.play(sped_up)
        self.wait()

Manim Community v0.18.0

In [36]:
rust_code = "impl From<u64> for Health {"
token_replacements = "impl TryFrom<u64> for Health {"

In [24]:
from dataclasses import dataclass
@dataclass
class CodeVGroup:
    raw: str
    vg: VGroup
    rendered: bool

def tokenize_from_str(s: str, language="rust"):
    tokens = s.split()
    code = Code(code=s, language=language, insert_line_no=False)

    tokenized_code = list(filter(lambda x: isinstance(x, VGroup), tokenize(code)))
    vgmap = [CodeVGroup(raw, vg, False) for raw, vg in zip(tokens, tokenized_code)]

    return vgmap

In [37]:
def common_tokens(a: str, b: str):
    l_stack = a.split()
    r_stack = b.split()
    l_ptr = 0
    r_ptr = 0
    l_list = []
    r_list = []
    
    while l_ptr < len(l_stack) - 1 or r_ptr < len(r_stack) - 1:
        if l_stack[l_ptr] == r_stack[r_ptr]:
            l_list.append(l_ptr)
            r_list.append(r_ptr)
            l_ptr += 1
            r_ptr += 1
            continue
        old_r_ptr = r_ptr
        while r_ptr < len(r_stack):
            if r_stack[r_ptr] == l_stack[l_ptr]:
                l_list.append(l_ptr)
                r_list.append(r_ptr)
                l_ptr += 1
                r_ptr += 1
                break
            r_ptr += 1
        if r_ptr == len(r_stack):
            r_ptr = old_r_ptr
            l_ptr += 1
    return l_list, r_list

In [38]:
common_tokens(rust_code, token_replacements)

([0, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13],
 [0, 2, 3, 4, 9, 10, 11, 12, 15, 16, 17, 18])

In [ ]:
%%manim -qm -v WARNING CodeBlock

class CodeBlock(Scene):
    def construct(self):
        code = tokenize(Code(code=rust_code, language="rust", insert_line_no=False))
        replacement_code = tokenize(Code(code=token_replacements, language="rust", insert_line_no=False))
        for token in code:
            anim = Write(token)
            sped_up = ChangeSpeed(
                anim,
                speedinfo={0:3},
                rate_func=linear
            )
            self.play(sped_up)

        tokenized_code = list(filter(lambda x: isinstance(x, VGroup), code))
        tokenized_rep = list(filter(lambda x: isinstance(x, VGroup), replacement_code))

        l_list, r_list = common_tokens(rust_code, token_replacements)

        for i, x in enumerate(tokenized_code):
            if i not in l_list:
                anim = Unwrite(x)
                sped_up = ChangeSpeed(
                    anim,
                    speedinfo={0:3},
                    rate_func=linear
                )
                self.play(sped_up)

        for a, b in zip(l_list, r_list):
            anim = Transform(tokenized_code[a], tokenized_rep[b])
            sped_up = ChangeSpeed(
                anim,
                speedinfo={0:3},
                rate_func=linear
            )
            self.play(sped_up)

        for i, x in enumerate(tokenized_rep):
            if i not in r_list:
                anim = Write(x)
                sped_up = ChangeSpeed(
                    anim,
                    speedinfo={0:3},
                    rate_func=linear
                )
                self.play(sped_up)
        self.wait()

Manim Community v0.18.0